<a href="https://colab.research.google.com/github/SJin765/class_AI4dl/blob/main/Team_project/Base%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# 현재 작업 디렉토리의 파일 경로
file_path = os.path.join(os.getcwd(), 'drive', 'MyDrive','open.zip')
# 경로 출력
print(file_path)

In [ ]:
## 자기 컴퓨터 환경따라 하기

import zipfile

zip_file_path = '/content/drive/MyDrive/open.zip'
extract_path = '/content/drive/MyDrive/extract/folder'

# 압축 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    # 압축 파일 내 폴더 목록 확인
    folder_list = [name for name in zip_file.namelist() if name.endswith('/')]

    # 폴더 내용물 확인
    for folder_name in folder_list:
        print(f"Folder: {folder_name}")
        file_list = zip_file.namelist()

        # 폴더 내 파일 목록 확인
        for file_name in file_list:
            if file_name.startswith(folder_name) and not file_name.endswith('/'):
                print(f"File: {file_name}")

    # 압축 파일 해제
    zip_file.extractall(extract_path)

# Import

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision
import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.auto import tqdm

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter

In [ ]:
CFG = {
    'NUM_CLASS':34,
    'IMG_SIZE':512, #512고정
    'EPOCHS':10,
    'LR':3e-4, #Learning rate
    'BATCH_SIZE':16,
    'SEED':41
}

# Fixed random-seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# Visualization

In [ ]:
def draw_boxes_on_image(image_path, annotation_path):
    # 이미지 불러오기
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # txt 파일에서 Class ID와 Bounding Box 정보 읽기
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        values = list(map(float, line.strip().split(' ')))
        class_id = int(values[0])
        x_min, y_min = int(round(values[1])), int(round(values[2]))
        x_max, y_max = int(round(max(values[3], values[5], values[7]))), int(round(max(values[4], values[6], values[8])))

        # 이미지에 바운딩 박스 그리기
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        cv2.putText(image, str(class_id), (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 이미지와 바운딩 박스 출력
    plt.figure(figsize=(25, 25))
    plt.imshow(image)
    plt.show()

In [ ]:
# 파일 경로 설정 - 첫번째 파일
train_path = '/content/drive/MyDrive/extract/folder/train'
img_name = '/syn_00000.png'
ant_name = '/syn_00000.txt'
image_file = train_path + img_name
annotation_file = train_path + ant_name
# 함수 실행
draw_boxes_on_image(image_file, annotation_file)

In [ ]:
# 파일 경로 설정 - 두번째 파일
train_path = '/content/drive/MyDrive/extract/folder/train'
img_name = '/syn_00001.png'
ant_name = '/syn_00001.txt'
image_file = train_path + img_name
annotation_file = train_path + ant_name
# 함수 실행
draw_boxes_on_image(image_file, annotation_file)

# Custom dataset


In [ ]:
#collate_fn : 주어진 이미지 - target box, target label추출
def collate_fn(batch): #Pytorch의 dataloader에서 batch를 처리하기 위한 함수
    images, targets_boxes, targets_labels = tuple(zip(*batch))
    images = torch.stack(images, 0)
    targets = []
   
    for i in range(len(targets_boxes)):
        target = {
            "boxes": targets_boxes[i],
            "labels": targets_labels[i]
        }
        targets.append(target)

    return images, targets